# Allele Sequence Confirmation and QC Results

We have conducted multiple rounds of allele sequencing confirmation and quality-control checks to make sure that we identify the true hits of mislocalization and morphological changes for the genetic variants we test. This notebook keeps track of the sequence confirmation results we obtained from different sequencing centers and conducts the QC and data clean-up process for the platemap we have on the AWS s3 Cell Painting Gallery.

In [95]:
import os
import glob
import polars as pl

BATCHES = "2024_01_23_Batch_7 2024_02_06_Batch_8 2024_12_09_Batch_11 2024_12_09_Batch_12 2025_01_27_Batch_13 2025_01_28_Batch_14 2025_03_17_Batch_15 2025_03_17_Batch_16"

control_alleles_dict = {
    "TC": ["EGFP"],
    "NC": ["RHEB", "MAPK9", "PRKACB", "SLIRP"],
    "PC": ["ALK", "ALK_Arg1275Gln", "PTK2B"],
    "cPC": ["KRAS", "PTK2B", "GHSR", "ABL1", "BRD4", "OPRM1", "RB1", 
            "ADA", "WT PMP22", "LYN", "TNF", "CYP2A6", "CSK", "PAK1", 
            "ALDH2", "CHRM3", "KCNQ2", "ALK T1151M", "PRKCE", "LPAR1", "PLP1"
    ]
}

In [ ]:
allele_collection_seq_confirmation = pl.read_parquet(
    "../inputs/allele_seq_confirm/VarChAMP_Allele_Collection_1Perc_ClinVar_dbNSFP_pLDDT_RSA_G2P_OMIM_MOI.parquet",
)
# allele_collection_seq_confirmation.filter(
#     pl.col("mislocalization_seq_final")=="Y",
# ).select(
#     pl.col("symbol"),
#     pl.col("gene_variant"),
#     pl.col("aa_change"),
#     pl.col("mislocalization_seq_final"),
# )#.unique(subset=["mislocalization_seq_final"])

## Sequence confirmation as FALSE
allele_to_exclude = allele_collection_seq_confirmation.filter(
    pl.col("mislocalization_seq_final")=="N",
    pl.col("entry_sequenced")==1,
    pl.col("mislocalization_sequenced")==1
)
## Imaging sequence confirmation
img_seq_result_df = pl.read_csv(
    "../inputs/allele_seq_confirm/imaging_analyses_mislocalization_summary_clinvar_TT_update_final_20251120.csv"
)
allele_to_exclude_2 = img_seq_result_df.filter(
    pl.col("seq_result").is_in([3,4,5])
).with_columns(
    pl.col("gene_allele").alias("gene_variant")
)
## incorrect reference allele
ref_allele_to_exclude = pl.read_csv(
    "../inputs/allele_seq_confirm/incorrect_ref_allele_email_thread_AugSepOct.csv"
)
## verified all these results should be filtered out
# allele_collection_seq_confirmation.filter(
#     pl.col("gene_variant").is_in(["COQ8A_His85Gln","MLH1_Val716Met","FH_Gln376Pro"])
# ).select(
#     pl.col(["gene_variant"]+[c for c in allele_collection_seq_confirmation.columns if "entry" in c or "misloc" in c])
# )

var_alleles_to_drop = allele_to_exclude["gene_variant"].to_list() + allele_to_exclude_2["gene_variant"].to_list()
ref_alleles_to_drop = ref_allele_to_exclude["Orig_Gene"].to_list()
print("QC failed alleles:", len(alleles_to_drop+ref_alleles_to_drop))

QC failed alleles: 475


## 1. Loading platemaps by batch

In [123]:
COLS_TO_DROP = ["clinvar_clinical_significance", "clinvar_allele_ID", "hgmd_ID", "control_type", "gene", "allele", "final_gene_allele", "gene_symbol"]
COLS_TO_NULL = ["symbol", "gene_allele", "node_type", "orf_id_wt", "ccsb_mutation_id", "nt_change_cdna", "aa_change"]

for batch_dir in BATCHES.split():
    for raw_meta_data in glob.glob(f"../inputs/platemaps/{batch_dir}/platemap/*.txt"):
        print(raw_meta_data)
        plate_map_df = pl.read_csv(raw_meta_data, separator="\t")
        ## These columns are always nulls or deduplicated, remove for clarity
        plate_map_df = plate_map_df.drop(set(COLS_TO_DROP) & set(plate_map_df.columns))
        ## Clean up the symbol and gene_allele names to remove weird chars
        plate_map_df = plate_map_df.with_columns(
            pl.col("symbol").str.replace_all(r"[^a-zA-Z0-9]", "").alias("symbol"),
            pl.col("gene_allele").str.replace_all(r"[^a-zA-Z0-9_]", "").alias("gene_allele"),
        )
        display(plate_map_df.filter(
            pl.col("gene_allele").is_in(var_alleles_to_drop)
        ))
        display(plate_map_df.filter(
            pl.col("gene_allele").is_in(ref_alleles_to_drop)
        ))
        ## Drop the alleles
        plate_map_df = plate_map_df.with_columns([
            pl.when(pl.col("gene_allele").is_in(var_alleles_to_drop))
            .then(None)
            .otherwise(pl.col(c))
            .alias(c)
            for c in COLS_TO_NULL
        ])
        ## For batch 11-12 and 15-16, correct some wrong references too
        if "2024_12" in batch_dir or "2025_03" in batch_dir:
            plate_map_df = plate_map_df.with_columns([
                pl.when(pl.col("gene_allele").is_in(ref_alleles_to_drop))
                .then(None)
                .otherwise(pl.col(c))
                .alias(c)
                for c in COLS_TO_NULL
            ])
        display(plate_map_df.filter(
            pl.col("gene_allele").is_in(var_alleles_to_drop)
        ))
        display(plate_map_df.filter(
            pl.col("gene_allele").is_in(ref_alleles_to_drop)
        ))
        
        output_metadata_qc_file = raw_meta_data.replace("../inputs/platemaps", "../outputs/corrected_platemaps")
        output_dir = '/'.join(output_metadata_qc_file.split('/')[:-1])        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)

        plate_map_df.write_csv(output_metadata_qc_file, separator="\t")

../inputs/platemaps/2024_01_23_Batch_7/platemap/B7A1R1_P3.txt


plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter
str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,f64,str,i64,i64,i64,i64,i64,i64,str
"""B7A1R1_P3""","""M03""","""MLH1""","""MLH1_Asn38His""","""M03""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",2713.0,467.0,"""112A>C""","""Asn38His""",1.0,3,7,8,null,527.0,null,3,1,2,7,13,3,"""M"""
"""B7A1R1_P3""","""G05""","""MLH1""","""MLH1_Ala29Gly""","""G05""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",2713.0,451.0,"""86C>G""","""Ala29Gly""",1.0,3,7,8,null,527.0,null,3,1,3,4,7,5,"""G"""
"""B7A1R1_P3""","""I05""","""MLH1""","""MLH1_Cys39Phe""","""I05""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",2713.0,473.0,"""116G>T""","""Cys39Phe""",1.0,3,7,8,null,527.0,null,3,1,3,5,9,5,"""I"""
"""B7A1R1_P3""","""M05""","""MLH1""","""MLH1_Val716Met""","""M05""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",2713.0,1015.0,"""2146G>A""","""Val716Met""",1.0,3,7,8,null,527.0,null,3,1,3,7,13,5,"""M"""
"""B7A1R1_P3""","""O05""","""MLH1""","""MLH1_Pro747Ser""","""O05""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",2713.0,1033.0,"""2239C>T""","""Pro747Ser""",1.0,3,7,8,null,527.0,null,3,1,3,8,15,5,"""O"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B7A1R1_P3""","""B08""","""PRPS1""","""PRPS1_Leu129Ile""","""B08""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",3731.0,201248.0,"""385C>A""","""Leu129Ile""",1.0,3,7,8,null,527.0,null,3,4,4,1,2,8,"""B"""
"""B7A1R1_P3""","""H12""","""PYGL""","""PYGL_Asn632Ile""","""H12""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",55246.0,219175.0,"""1895A>T""","""Asn632Ile""",1.0,3,7,8,null,527.0,null,3,4,6,4,8,12,"""H"""
"""B7A1R1_P3""","""H14""","""RAB33B""","""RAB33B_Lys46Gln""","""H14""","""B7A1R1_P3""","""B8A1R2_P3""","""allele""",71354.0,205775.0,"""136A>C""","""Lys46Gln""",1.0,3,7,8,null,527.0,null,3,4,7,4,8,14,"""H"""


plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter
str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,f64,str,i64,i64,i64,i64,i64,i64,str


plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter
str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,f64,str,i64,i64,i64,i64,i64,i64,str


plate_map_name,well_position,symbol,gene_allele,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,ccsb_mutation_id,nt_change_cdna,aa_change,allele_set,imaging_plate,batch1,batch2,clone_ID,vector,notes,plate,quad,c96,r96,r384,c384,r384_letter
str,str,str,str,str,str,str,str,f64,f64,str,str,f64,i64,i64,i64,str,f64,str,i64,i64,i64,i64,i64,i64,str


...

../inputs/platemaps/2025_03_17_Batch_16/platemap/B16A1A2P1_R1.txt


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,sequencing_confidence
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str
"""B16A1A2P1_R1""","""E01""","""AP2S1""","""AP2S1_Arg15His""","""VUSmut_GDEh06""","""G05""",39.0,"""FVrry_P05_Q01""","""C01""",3,"""E01""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4983.0,"""44G>A""","""Arg15His""",215316.0,null
"""B16A1A2P1_R1""","""M09""","""CLDN19""","""CLDN19_Gln57Glu""","""VUSmut_GDEh05""","""H02""",16.0,"""FVrry_P05_Q01""","""G05""",39,"""M09""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",9634.0,"""169C>G""","""Gln57Glu""",216880.0,null
"""B16A1A2P1_R1""","""E11""","""COQ8A""","""COQ8A_Gly549Ser""","""VUSmut_GDEh06""","""B01""",2.0,"""FVrry_P05_Q01""","""C06""",43,"""E11""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4043.0,"""1645G>A""","""Gly549Ser""",201419.0,null
"""B16A1A2P1_R1""","""G11""","""COQ8A""","""COQ8A_His85Gln""","""FV_reLR_02""","""C10""",75.0,"""FVrry_P05_Q01""","""D06""",44,"""G11""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4043.0,"""255T>G""","""His85Gln""",900203.0,"""low"""
"""B16A1A2P1_R1""","""I11""","""COQ8A""","""COQ8A_His80Tyr""","""VUSmut_GDEh12""","""G03""",23.0,"""FVrry_P05_Q01""","""E06""",45,"""I11""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4043.0,"""238C>T""","""His80Tyr""",900212.0,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""B16A1A2P1_R1""","""N21""","""TMEM43""","""TMEM43_Thr277Ser""","""VUSmut_GDEh05""","""B06""",42.0,"""FVrry_P05_Q03""","""G11""",87,"""N21""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",3438.0,"""829A>T""","""Thr277Ser""",201180.0,null
"""B16A1A2P1_R1""","""H23""","""TMEM43""","""TMEM43_Arg312Trp""","""VUSmut_GDEh07""","""G10""",79.0,"""FVrry_P05_Q03""","""D12""",92,"""H23""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",3438.0,"""934C>T""","""Arg312Trp""",201181.0,null
"""B16A1A2P1_R1""","""B04""","""TPM1""","""TPM1_Asn279His""","""VUSmut_GDEh08""","""B12""",90.0,"""FVrry_P05_Q04""","""A02""",9,"""B04""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""allele""",4723.0,"""835A>C""","""Asn279His""",201750.0,null


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,sequencing_confidence
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str
"""B16A1A2P1_R1""","""K05""","""BFSP2""","""BFSP2""","""DisWTGDEh03""","""E10""",77.0,"""FVrry_P05_Q01""","""F03""",22,"""K05""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",53737.0,null,null,null,null
"""B16A1A2P1_R1""","""A11""","""COQ8A""","""COQ8A""","""DisWTGDEh02""","""F09""",70.0,"""FVrry_P05_Q01""","""A06""",41,"""A11""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",4043.0,null,null,null,null
"""B16A1A2P1_R1""","""C04""","""IMPDH1""","""IMPDH1""","""FV_reLR_01""","""H04""",32.0,"""FVrry_P05_Q02""","""B02""",10,"""C04""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",71077.0,null,null,null,"""low """
"""B16A1A2P1_R1""","""J08""","""TPM3""","""TPM3""","""DisWTGDEh03""","""F07""",54.0,"""FVrry_P05_Q04""","""E04""",29,"""J08""","""B15A1A2P1_R1""","""B16A1A2P1_R2""","""disease_wt""",5712.0,null,null,null,null


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,sequencing_confidence
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str


plate_map_name,well_position,symbol,gene_allele,source_plate,source_well,source_numb,dest_plate,dest_well,dest_numb,imaging_well,imaging_plate_R1,imaging_plate_R2,node_type,orf_id_wt,nt_change_cdna,aa_change,ccsb_mutation_id,sequencing_confidence
str,str,str,str,str,str,f64,str,str,i64,str,str,str,str,f64,str,str,f64,str
